# Exploiting and transforming Ardupilot-Log files 

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
from pathlib import Path
import pandas as pd

module_path = str(Path.cwd().parents[0] / "src")
#module_path = str(Path.cwd(__file__).parents[0] / "py")

if module_path not in sys.path:
    sys.path.append(module_path)

from parser.DataflashParser import read_from_log, write_csv, interpolate_data
from data_aggregation.ardupilot_to_pandas import run_aggregation

In [7]:
input_data_folder_base = Path("../data/raw_data")
output_data_folder_base = Path("../data/preproc_data/")
ardupilot_sub_folder = Path("./ardupilot")

# folder structure containing all ardupilot data files
ardupilot_input_data_folder = Path.cwd() / input_data_folder_base / ardupilot_sub_folder 
print(f"Reading raw data files from \n    {ardupilot_input_data_folder}")

ardupilot_output_data_folder = Path.cwd() / output_data_folder_base / ardupilot_sub_folder 
print(f"Writing preprocessed pandas files to \n    {ardupilot_output_data_folder}")


Reading raw data files from 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/data_aggregation/notebooks/../data/raw_data/ardupilot
Writing preprocessed pandas files to 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/data_aggregation/notebooks/../data/preproc_data/ardupilot


In [10]:
# Collecting position attributes for assignments to ardupilot-Dataframes

locations = {
    'Balbina': ["00000038.log", "00000039.log", "00000040.log", "00000041.log", "00000042.log"],
    'Jandira': ["00000043.log", "00000044.log", "00000045.log"],
    'Caldeirao': ["00000046.log", "00000047.log"],
}

def assign_location(file_name):
    for location in locations:
        if file_name in locations[location]:
            return location
    return "not known"

In [11]:
file_list = list(ardupilot_input_data_folder.rglob("*.log"))
print(f"Found {len(file_list)} raw files.")

for index, file in enumerate(file_list):
    place = assign_location(file.name)
    print(f"({index}/{len(file_list)-1}) - {file.name} / {place}")
    df=run_aggregation(file)
    df['experiment_location'] = place

    df.drop(['index', 'datetime_UTC'], axis=1, inplace=True)
    
    filename = df.index.min().strftime('%d%m%y-%H%M%S-AP')
    result_file = ardupilot_output_data_folder / Path(filename+".p")   
    print(f"    ... saving to { Path(filename+'.p')}")
    df.to_pickle(result_file)

Found 10 raw files.
(0/9) - 00000039.log / Balbina
    ... extracting estimated positional information from AHR2
    ... extracting time stamps and positions from GPS
    ... aggregate depth information from RFND
    ... saving to 010323-182204-AP.p
(1/9) - 00000038.log / Balbina
    ... extracting estimated positional information from AHR2
    ... extracting time stamps and positions from GPS
    ... aggregate depth information from RFND
    ... saving to 010323-134619-AP.p
(2/9) - 00000040.log / Balbina
    ... extracting estimated positional information from AHR2
    ... extracting time stamps and positions from GPS
    ... aggregate depth information from RFND
    ... saving to 010323-212223-AP.p
(3/9) - 00000042.log / Balbina
    ... extracting estimated positional information from AHR2
    ... extracting time stamps and positions from GPS
    ... aggregate depth information from RFND
    ... saving to 020323-132955-AP.p
(4/9) - 00000041.log / Balbina
    ... extracting estimated 